# DSCI-100 Individual Planning Stage of Project

**Name:** Kyle Li
**Section:** 003
**Group:** 9
**Date:** Nov. 11, 2025




In [1]:
## Necessary Components to Preload

library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

# (1) Data Description:

<u><b>What and Why:</u></b> The data we are provided in this project are the _players.csv_ and _sessions.csv_ datasets which originate from a Minecraft server run by a research group in Computer Science at UBC. The goal of this Minecraft server is to ultimately use player input data to train AI. However, I will only be analyzing the _players.csv_ as the predictive question I aim to answer doesn't require the _sessions.csv_ dataset. The specificity of this question will be expanded upon in the subsequent section. As such, in this section, we are firstly going to identify and define the characteristics of _players.csv_ and then summarize its statistics. It is important to note that the _players.csv_ data presented was gathered via self-reporting during server signup and compiled into a csv by the researchers. 


<u><b>Overview of Players Dataset:</u></b>
| Dataset | Number of Observations | Number of Variables | Represents |  
|:-----|:---:|:---:|:-----|
| players.csv | 196 | 7 | Data and list of all unique players involved with the server. |

<u><b>Player Dataset:</u></b>

**a) Variables**
| Variable | Type | Short Description |
|:-----|:-----|:-----|
| experience | Character | Self-reported level of experience with Minecraft. This ranged from Amateur -> Reguler -> Veteran -> Pro |
| subscribe | Logical (T/F) | Whether a player had suscribed to a game-related newsletter (yes = True, no = False) |
| hashedEmail | Character | Unique identifier of each player (anonymizing their emails via hashing) |
| played_hours | Numeric (dbl) | Hours spent on the Minecraft server for each participant |
| name | Character | First names (not gamertag) of each individual participating|
| gender | Character | Gender of each individual participating |
| Age | Numeric (dbl) | Age in years of each individual participating |

**b) Summary Statistics**
| Variable | Value (round to 2 decimals if possible) | 
|:-----|:-----:|
| Mean Age | 21.14 |
| Youngest Age | 9 |
| Oldest Age | 58 |
| Standard Deviation of Age | 7.39 | 
| Mean Hours Played | 5.85 |
| Highest Number of Hours Played | 223.1|
| Lowest Number of Hours Played | 0 |
| Standard Deviation of Hours Played | 28.36 | 
| Number of Players | 196 | 
| Proportion Subscribed (%) | 73.47 | 

Note: Since we know that each player (196 total) has a unique identifier and a first name, we also know that the number of hashed emails and names also equal to 196. Furthermore, for the sake of simplicity, I will not be presenting gender or experience proportions in the Summary Statistics table.

<u><b>Possible Concerns/Potential Issues:</u></b>
- Players may have overestimated or underestimated their abilities when stating their experience 
- Hours played may not be as exact since it seems to be rounded to 1 decimal in the dataset thus, those who log in for a few minutes may be reported as 0hrs
- Many players show 0 hours played which may indicate logging errors or lack of participation which may skew results
- People involved with this server may have submitted incorrect information e.g. gender, age, game-related newsletter subscription, etc.
- Factors outside of Minecraft may have influenced their ability to play on the server such as heavy workloads, access to a device that can properly run Minecraft, less of a workload which may lead to an abnormal increase of playtime, and etc.